In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from re import sub
import pandas as pd
import timeit

In [2]:
with open('url.txt', 'r') as f:
    url = f.read()
    # Load Fist PageOpen connection and download HTML page
    uClient = uReq(url)
    page_soup = soup(uClient.read(), "html.parser")
    uClient.close()

In [3]:
i = 0
success = 0
errors = 0
pages_scrapped = 0
data_set = list()
page = 0

start_time = timeit.default_timer()

while True:
    try:
        # Get only car Data From the HMTL CODE
        containers = page_soup.findAll('li', {"class": "card-item"})

        # Loop for all cars in current page
        for row in containers:

            car = dict()

            # Get ad url
            ad_url = row.a['href']

            car['ad_name'] = row.find('p', {'class': 'card__body-title'}).text

            # Price
            car['price'] = row.find('p', {'class': 'card__price'}).text

            # Get ad and store it in a STR
            uClient = uReq(ad_url)
            ad_soup = str(soup(uClient.read(), "html.parser"))
            uClient.close()
            try:
                # Extract Ad key details dictionary
                start = ad_soup.find('"displayAttributes"') 
                end = ad_soup.find('"publisherName"')
                ad_soup = ad_soup[start:end]
                start = ad_soup.find(r'[{"')
                end = ad_soup.find(r'}}],')
                ad_soup = ad_soup[start:end + 3]
                # Convert String Dic to Dictionary
                ad_soup = ad_soup.replace('false', 'False')
                ad_soup_dic = eval(ad_soup)
                for row in ad_soup_dic:
                    car[str(row["name"])] = row['value']

                data_set.append(car)
                success += 1

                #print(f'{i} -> SUCCESS')

            except:
                errors += 1
                #print(f'{i} -> ERROR')
                pass

            i += 1

        # add 28 to load next set
        page += 28

        # Load Next Page
        page_url = url + '?start=' + str(page)

        # Open connection and download HTML page
        uClient = uReq(page_url)
        page_soup = soup(uClient.read(), "html.parser")
        uClient.close()
        pages_scrapped += 1
        print(f'Pages Scrapped: {pages_scrapped}')

    except:
        errors += 1
        # add 28 to load next set
        page += 28

        # Load Next Page
        page_url = url + '?start=' + str(page)

        pass
    
    if pages_scrapped > 1000:
        break
    if pages_scrapped % 100 == 0:
        dframe = pd.DataFrame(data_set)
        file_name = 'car_data_p' + str(pages_scrapped) + '.csv'
        dframe.to_csv(file_name)
        
        
end_time = timeit.default_timer()

# Print Results of scrapping
print(f'Run Time: {end_time - start_time} secods')
print(f'Pages Scrapped: {pages_scrapped}')
print(f'Success: {success}')
print(f'Errors: {errors}')

Pages Scrapped: 1
Pages Scrapped: 2
Pages Scrapped: 3
Pages Scrapped: 4
Pages Scrapped: 5
Pages Scrapped: 6
Pages Scrapped: 7
Pages Scrapped: 8
Pages Scrapped: 9
Pages Scrapped: 10
Pages Scrapped: 11
Pages Scrapped: 12
Pages Scrapped: 13
Pages Scrapped: 14
Pages Scrapped: 15
Pages Scrapped: 16
Pages Scrapped: 17
Pages Scrapped: 18
Pages Scrapped: 19
Pages Scrapped: 20
Pages Scrapped: 21
Pages Scrapped: 22
Pages Scrapped: 23
Pages Scrapped: 24
Pages Scrapped: 25
Pages Scrapped: 26
Pages Scrapped: 27
Pages Scrapped: 28
Pages Scrapped: 29
Pages Scrapped: 30
Pages Scrapped: 31
Pages Scrapped: 32
Pages Scrapped: 33
Pages Scrapped: 34
Pages Scrapped: 35
Pages Scrapped: 36
Pages Scrapped: 37
Pages Scrapped: 38
Pages Scrapped: 39
Pages Scrapped: 40
Pages Scrapped: 41
Pages Scrapped: 42
Pages Scrapped: 43
Pages Scrapped: 44
Pages Scrapped: 45
Pages Scrapped: 46
Pages Scrapped: 47
Pages Scrapped: 48
Pages Scrapped: 49
Pages Scrapped: 50
Pages Scrapped: 51
Pages Scrapped: 52
Pages Scrapped: 53
Pa

In [4]:
df = pd.DataFrame(data_set)

In [5]:
df.dtypes

ad_name           object
price             object
make              object
model             object
year              object
mileage           object
fuelType          object
transmission      object
bodyType          object
seats             object
engine            object
roadTax           object
NCT               object
previousOwners    object
country           object
colour            object
numDoors          object
maxSpeed          object
mpgCombined       object
cylinders         object
noughtToSixty     object
tankCapacity      object
vehicleHeight     object
vehicleLength     object
vehicleWidth      object
dtype: object

In [6]:
df.describe()

,ad_name,price,make,model,year,mileage,fuelType,transmission,bodyType,seats,...,colour,numDoors,maxSpeed,mpgCombined,cylinders,noughtToSixty,tankCapacity,vehicleHeight,vehicleLength,vehicleWidth
count,32755,32755,32755,32755,32755,32755,32755,32755,32755,32755,...,32755,32755,12,12,12,12,12,12,12,12
unique,23443,2838,75,718,54,11462,9,13,52,17,...,15,17,12,11,2,12,9,8,10,10
top,Car,No Price,Volkswagen,Golf,2017,,Diesel,Manual,Hatchback,,...,Black,5,170 km/h,37 mpg,4,9.4 sec,62 litres,"1,427 mm","4,526 mm","1,728 mm"
freq,118,987,3858,1271,4424,1537,20947,21114,12679,18658,...,6917,15453,1,2,11,1,2,4,2,2


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32755 entries, 0 to 32754
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ad_name         32755 non-null  object
 1   price           32755 non-null  object
 2   make            32755 non-null  object
 3   model           32755 non-null  object
 4   year            32755 non-null  object
 5   mileage         32755 non-null  object
 6   fuelType        32755 non-null  object
 7   transmission    32755 non-null  object
 8   bodyType        32755 non-null  object
 9   seats           32755 non-null  object
 10  engine          32755 non-null  object
 11  roadTax         32755 non-null  object
 12  NCT             32755 non-null  object
 13  previousOwners  32755 non-null  object
 14  country         32755 non-null  object
 15  colour          32755 non-null  object
 16  numDoors        32755 non-null  object
 17  maxSpeed        12 non-null     object
 18  mpgCom

In [8]:
df

,ad_name,price,make,model,year,mileage,fuelType,transmission,bodyType,seats,...,colour,numDoors,maxSpeed,mpgCombined,cylinders,noughtToSixty,tankCapacity,vehicleHeight,vehicleLength,vehicleWidth
0,Toyota Yaris 1.5 Hybrid Luxury SAT NAV Privacy Gl,"€16,249",Toyota,Yaris,2017,"44,810 km",Hybrid,Automatic,Hatchback,,...,Silver,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MINI ONE 1.6 DIESEL LIKE NEW,"€7,250",Mini,ONE,2011,"148,121 km",Diesel,,Hatchback,4,...,Orange,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BMW 520D SE 4dr STEP AUTO 2013: NCT & TAXED-2022,"€11,650",BMW,5-Series,2013,"146,157 mi",Diesel,Automatic,Saloon,5,...,Black,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mercedes Viano 2009,"€6,700",Mercedes-Benz,Viano,2009,"266,291 mi",Diesel,Automatic,Cabriolet,7,...,Silver,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11 Peugeot RCZ Coupe,"€8,950",Peugeot,RCZ,2011,"94,500 mi",Diesel,Manual,Coupe,4,...,Grey,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32750,15 BMW 520D 190BHP AUTOMATIC MODEL HUGE SPEC,"€16,900",BMW,5-Series,2015,"129,000 mi",Diesel,Automatic,Saloon,5,...,Black,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32751,171 VOLKSWAGEN CADDY MAXI LIFE AUTOMATIC DSG M...,"€19,900",Volkswagen,Caddy,2017,"66,000 mi",Diesel,Automatic,MPV,7,...,Grey,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32752,Nissan Micra 0.9 SV 4DR HI Spec,"€12,950",Nissan,Micra,2017,"28,250 km",Petrol,Manual,Hatchback,,...,Grey,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32753,Nissan Micra SV High Spec,"€12,950",Nissan,Micra,2017,"46,211 km",Petrol,Manual,Hatchback,,...,Grey,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.to_csv('car_data2.csv')

In [6]:
100 % 10

0